In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import shutil

class SMT_test:
    chart = None
    def __init__(self, benchmarks_path='../../', chart_path='../charts', result_path='../results'):
        self.benchmarks_path = benchmarks_path
        self.chart_path = chart_path
        self.result_path = result_path

    def init_chart(self, folder_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_folder_path = f'{self.benchmarks_path}{folder_path}'
        self.get_info(full_folder_path)
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)
    
    def get_info(self, folder_path):
        benchmarks = os.listdir(folder_path)
        for file in benchmarks:
            path = os.path.join(folder_path, file)
            if os.path.isdir(path):
                self.get_info(path)
            else:
                try:
                    with open(path) as f:
                        vars = 0
                        assertions = 0
                        status = 'none'
                        line = f.readline()
                        while line:
                            if line[:3] == '(de':
                                vars += 1
                            elif line[:3] == '(as':
                                assertions += 1
                            elif line[11:17] == 'status':
                                status = line.split(' ')[2][:-2]
                            line = f.readline()
                        category = path.split('/')[-2]
                        path = path.replace(self.benchmarks_path, '')
                        self.chart.loc[len(self.chart.index)] = [path, category, vars, assertions, status]
                except:
                    print(path)

    def move_files_to_folder(self, path_list, destination_folder):
        cnt = 0
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)
        for path in path_list:
            file_name = os.path.basename(path)
            destination_path = os.path.join(destination_folder, file_name)
            if os.path.isfile(path):
                shutil.move(path, destination_path)
            elif not os.path.isfile(destination_path):
                cnt += 1
                print(f'{path} is not a file')
        print(f"count: {cnt}")   

    def get_need_path(self, solver, log_path, read_path, save_path):
        full_res_path = f'{self.result_path}/{solver}/{log_path}'
        results = os.listdir(full_res_path)
        count = 0
        file_path_list = []
        for file in tqdm(results, desc=full_res_path):
            with open(os.path.join(full_res_path, file)) as f:
                try:
                    if solver == 'z3' or solver == 'z3pp_ls' or solver == 'z3pp':
                        while f.readline():
                            result = f.readline().rstrip()
                            # OMT-test only
                            if result == "sat":
                                f.readline()
                                f.readline()
                                f.readline()
                                f.readline()
                            else:
                                assert(result == "timeout" or result == "unknown" or result[:3] == "./s")
                                if result == "unknown":
                                    f.readline()
                                    f.readline()
                                    f.readline()
                                info = f.readline().rstrip()
                                if info == "unknown":
                                    info = f.readline().rstrip()
                                name = info.split(' : ')[0]
                                file_path = f'{read_path}/{name}'
                                file_path_list.append(file_path)
                                count += 1
                except:
                    print(file)
        self.move_files_to_folder(file_path_list, save_path)
        # with open(f'{save_path}/count.txt', 'w') as f:
        #     for path in file_path_list:
        #         f.write(path + '\n')

        print(f"count: {count}")

In [2]:
omt = SMT_test()
omt.get_need_path('z3pp_ls', 'OMT_LIA', '/home/hanrui', '../OMT_LIA')

../results/z3pp_ls/OMT_LIA:   0%|          | 0/192 [00:00<?, ?it/s]

count: 0
count: 318
